<a href="https://colab.research.google.com/github/Miftahul93/Pengembangan_Machine_Learning_Dicoding/blob/main/Lat_Transfer_Learning_Memproses_Dataset_Gambar_dari_Dunia_Nyata.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Transfer Learning**
Pada masalah computer vision, kesuksesan model yang kita kembangkan secara signifikan dipengaruhi oleh jumlah data yang tersedia. Jumlah data yang sedikit cenderung akan mengakibatkan model kita overfit sehingga prediksi dari model hasilnya tidaklah bagus. Pada submodul sebelumnya kita telah mempelajari salah satu teknik untuk mengatasi jumlah data yang terbatas dengan augmentasi gambar. Selain augmentasi gambar, ada teknik lain yang dapat kita pakai ketika data yang kita miliki untuk pengembangan model terbatas yaitu, transfer learning.

Ide dibalik transfer learning untuk masalah computer vision adalah, model yang telah dilatih pada dataset berukuran besar yang berisi gambar umum mampu dipakai sebagai model dasar yang membantu kita untuk mengenali fitur/bentuk-bentuk benda yang terdapat di dunia nyata. Kita dapat memanfaatkan fitur-fitur yang dipelajari ini tanpa harus melakukan pelatihan model dari awal sekali.

Sederhananya, transfer learning menggunakan model yang telah dilatih sebelumnya pada dataset lain, untuk dipakai pada dataset kita sendiri. Masih ingat? Layer-layer awal dan di tengah dari sebuah model bertugas untuk mengenali bentuk-bentuk pada gambar. Misalnya pada model pengenalan wajah manusia, pada layer pertama mungkin mengenali garis, layer kedua mengenali lingkaran, layer ketiga mengenali mata, dan layer keempat mampu mengenali wajah. Nah, layer-layer telah dilatih tersebut dapat kita manfaatkan untuk dipakai pada dataset lain misalnya seperti dataset pengenalan ekspresi manusia. 

Untuk melihat bagaimana efektifnya transfer learning, kita akan belajar menggunakan transfer learning pada dataset cheesman yang kita pelajari pada submodul sebelumnya.

In [1]:
!wget --no-check-certificate \
  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip \
  -O /tmp/Chessman-image-dataset.zip

--2021-08-03 10:05:45--  https://github.com/dicodingacademy/assets/raw/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip [following]
--2021-08-03 10:05:46--  https://raw.githubusercontent.com/dicodingacademy/assets/main/ml_pengembangan_academy/Chessman-image-dataset.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60684125 (58M) [application/zip]
Saving to: ‘/tmp/Chessman-image-dataset.zip’

/tmp/Chessman-image 100%[===================>]  57.87M   215MB/s   

Kemudian, kita buat direktori untuk dataset kita untuk digunakan pada ImageDataGenerator. Pada ImageDataGenerator gunakan augmentasi yang sama dengan latihan pada submodul sebelumnya.

In [2]:
import os
import zipfile
from tensorflow.keras.preprocessing.image import ImageDataGenerator
local_zip = '/tmp/Chessman-image-dataset.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/tmp')
zip_ref.close()
train_dir = os.path.join('/tmp/Chessman-image-dataset/Chess')
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=20,
                                   zoom_range=0.2,
                                   shear_range=0.2,
                                   fill_mode = 'nearest',
                                   validation_split=0.1) # Set validation split

Bagi dataset menjadi data training dan data validasi.

In [3]:
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(150,150),
    batch_size=8,
    class_mode='categorical',
    subset='training') # set as training data
validation_generator = train_datagen.flow_from_directory(
    train_dir, # same directory as training data
    target_size=(150,150),
    batch_size=16,
    class_mode='categorical',
    subset='validation')

Found 499 images belonging to 6 classes.
Found 52 images belonging to 6 classes.


Nah, disini kita mulai mengimplementasikan transfer learning. Untuk model yang kita pilih sebagai model dasar transfer learning adalah ResNet152V2. Model ResNet152V2 memiliki sebanyak 152 layer dan tersedia di library keras. Kita dapat memanfaatkan fitur-fitur yang telah dipelajari oleh model tersebut untuk dipakai model kita. 

Untuk mengimplementasikan transfer learning sangatlah mudah seperti kode di bawah. Kita hanya perlu menambahkan 2 buah baris kode berbeda. Layer pertama pada model kita adalah model yang kita pakai untuk transfer learning. Kita cukup memanggil kelas ResNet152V2 dan mengisi parameter sebagai berikut:


* Weight : ini adalah bobot atau parameter seperti yang telah dibahas pada kelas machine learning pemula. Untuk parameter weight kita mengisi nilai ‘imagenet’. Artinya kita ingin menggunakan model ResNet152V2 yang telah dilatih pada dataset imagenet. Imagenet adalah sebuah database raksasa yang berisi lebih dari 14 juta gambar.
* Include_top : parameter ini bernilai boolean. Maksud dari parameter ini apabila kita ingin tetap memakai layer terakhir/layer prediksi dari model resnet. Kita isi false karena kita memakai model resnet untuk memprediksi dataset chessman bukan imagenet.
* Input_tensor : sesuai namanya parameter ini menspesifikasikan ukuran dari input.

    
    

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.applications import ResNet152V2

model = tf.keras.models.Sequential([
  ResNet152V2(weights="imagenet", include_top=False, input_tensor=Input(shape=(150, 150, 3))),
  # tf.keras.layers.Dropout(0.4),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(512, activation='relu'),
  tf.keras.layers.Dense(256, activation='relu'),
  tf.keras.layers.Dense(6, activation='softmax')
])

model.layers[0].trainable = False

Lanjutkan dengan menentukan optimizer, loss, serta metrik yang ingin digunakan pada model.

In [6]:
model.compile(optimizer=tf.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics = ['accuracy'])

Terakhir kita dapat melakukan pelatihan model. 

In [7]:
history = model.fit(train_generator,
                    validation_data = validation_generator,
                    epochs=50,
                    verbose=2)

Epoch 1/50
63/63 - 52s - loss: 7.5872 - accuracy: 0.4409 - val_loss: 2.6770 - val_accuracy: 0.5769
Epoch 2/50
63/63 - 12s - loss: 1.7167 - accuracy: 0.6834 - val_loss: 0.8976 - val_accuracy: 0.7885
Epoch 3/50
63/63 - 12s - loss: 0.9207 - accuracy: 0.7715 - val_loss: 1.5894 - val_accuracy: 0.6538
Epoch 4/50
63/63 - 11s - loss: 1.0627 - accuracy: 0.7615 - val_loss: 1.7339 - val_accuracy: 0.6154
Epoch 5/50
63/63 - 11s - loss: 0.9478 - accuracy: 0.7876 - val_loss: 1.0475 - val_accuracy: 0.7500
Epoch 6/50
63/63 - 11s - loss: 0.6471 - accuracy: 0.8457 - val_loss: 1.3508 - val_accuracy: 0.6731
Epoch 7/50
63/63 - 12s - loss: 0.6389 - accuracy: 0.8377 - val_loss: 1.0652 - val_accuracy: 0.6923
Epoch 8/50
63/63 - 12s - loss: 0.7482 - accuracy: 0.8417 - val_loss: 0.7885 - val_accuracy: 0.7692
Epoch 9/50
63/63 - 11s - loss: 0.6092 - accuracy: 0.8377 - val_loss: 1.2058 - val_accuracy: 0.7308
Epoch 10/50
63/63 - 11s - loss: 0.3698 - accuracy: 0.8898 - val_loss: 1.0638 - val_accuracy: 0.7692
Epoch 11/